## Module 10 Assignment - Scraping a Website
* Author: Anagh Sharma
* version 2.0

We will be creating a web scraper to parse a table from the Charities Bureau Website. From the website: “All 
charitable organizations operating in New York State are required by law to register and file annual financial reports 
with the Attorney General's Office. This includes any organization that conducts charitable activities, holds property 
that is used for charitable purposes, or solicits financial or other contributions.”

In [1]:
!pip install selenium

In [2]:
!pip install webdriver_manager

In [3]:
!pip install awscli

  Obtaining dependency information for botocore==1.34.40 from https://files.pythonhosted.org/packages/4b/ca/f86918789bb290e9fb7573df4d496435c91196440479afc2b803f9d615bd/botocore-1.34.40-py3-none-any.whl.metadata
  Using cached botocore-1.34.40-py3-none-any.whl.metadata (5.7 kB)
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/83/37/395cdb6ee92925fa211e55d8f07b9f93cf93f60d7d4ce5e66fd73f1ea986/s3transfer-0.10.1-py3-none-any.whl.metadata
  Obtaining dependency information for colorama<0.4.5,>=0.2.5 from https://files.pythonhosted.org/packages/44/98/5b86278fbbf250d239ae0ecb724f8572af1c91f4a11edf4d36a206189440/colorama-0.4.4-py2.py3-none-any.whl.metadata
Using cached botocore-1.34.40-py3-none-any.whl (12.0 MB)
Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
   ---------------------------------------- 0.0/82.2 kB ? eta -:--:--
   ---------------------------------------- 82.2/82.2 kB 4.8 MB/s eta 0:00:00
  Attempting unins

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.34.40 which is incompatible.
boto3 1.24.28 requires botocore<1.28.0,>=1.27.28, but you have botocore 1.34.40 which is incompatible.
boto3 1.24.28 requires s3transfer<0.7.0,>=0.6.0, but you have s3transfer 0.10.1 which is incompatible.
pylint 2.16.2 requires colorama>=0.4.5; sys_platform == "win32", but you have colorama 0.4.4 which is incompatible.
wasabi 1.1.2 requires colorama>=0.4.6; sys_platform == "win32" and python_version >= "3.7", but you have colorama 0.4.4 which is incompatible.


In [4]:
###Load modules
#!pip install webdriver-manager
#!pip install awscli
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]') #identifies the location of the EIN element
inputElement.send_keys('0') #sends the "0" as the search value for EIN 
inputElement1 = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]').click() #instatiates the click of the search
sleep(4) #allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements(By.CSS_SELECTOR,'tr'):
        cols = df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
3,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
4,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
5,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
6,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
7,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
8,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
9,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY


In [7]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3://information-arch-vendor-bucket/' #specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_header_blank_row' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

C:\Users\anag\AppData\Local\Temp\ipykernel_3852\4047911759.py:9: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(filenames3, header=True, line_terminator='\n')


Successfull uploaded file to location:s3://information-arch-vendor-bucket/charities_bureau_scrape_header_blank_row20240413021040.csv


### Fixing Header Blank Row 

In [8]:
####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]') #identifies the location of the EIN element
inputElement.send_keys('0') #sends the "0" as the search value for EIN 
inputElement1 = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]').click() #instatiates the click of the search
sleep(4) #allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements(By.CSS_SELECTOR,'tr'):
    if len(row.find_elements(By.CSS_SELECTOR,'td')) != 0:
        cols = df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
1,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
5,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
6,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
7,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
8,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
9,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA


In [9]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3://information-arch-vendor-bucket/' #specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_header_blank_row_fixed' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

C:\Users\anag\AppData\Local\Temp\ipykernel_3852\1195453055.py:9: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(filenames3, header=True, line_terminator='\n')


Successfull uploaded file to location:s3://information-arch-vendor-bucket/charities_bureau_scrape_header_blank_row_fixed20240413022647.csv


## Part II. Update web-scraper to iterate all results and load csv file into S3 Bucket


In [19]:
from selenium.common.exceptions import NoSuchElementException
import warnings
warnings.filterwarnings("ignore")
####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]') #identifies the location of the EIN element
inputElement.send_keys('0') #sends the "0" as the search value for EIN 
inputElement1 = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]').click() #instatiates the click of the search
sleep(4) #allow for the page to load by adding a sleep element
#####CREATE DATE FRAME#####
#create empty dataframe

next_button = browser.find_element(By.XPATH,'/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a[text()="Next"]')
page_data = []  # Empty list to store data from each page
while True:
    try:
        next_button = browser.find_element(By.XPATH, '/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a[text()="Next"]')
        sleep(1)
        table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
        sleep(1)
        for row in table.find_elements(By.CSS_SELECTOR,'tr'):
            if len(row.find_elements(By.CSS_SELECTOR,'td')) != 0:
                page_data.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])
        next_button.click()
        sleep(6)
    except NoSuchElementException:
        table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
        sleep(1)
        for row in table.find_elements(By.CSS_SELECTOR,'tr'):
            if len(row.find_elements(By.CSS_SELECTOR,'td')) != 0:
                page_data.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])
        # Handle the case where "Next" button is not found (reached last page)
        print("Reached last page!")
        break

#update dataframe with header 
df = pd.DataFrame()
df = df.append(page_data, ignore_index=True)
df.columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"]
display(df) #let's have a look at the data before creating the CSV file and loading it into s3

Reached last page!


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
1,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
...,...,...,...,...,...,...
95,University of Virginia Health Foundtion,40-44-88,412097394,NFP,CHARLOTTESVILLE,VA
96,Violin Player,41-40-19,270773158,NFP,LONDONDERRY,NH
97,"William A. Epps Community Center, Inc.",40-91-11,861074714,NFP,STATEN ISLAND,NY
98,WORLD SOCIETY OF CZESTOCHOWA JEWS AND THEIR DE...,40-46-49,205101779,NFP,NEW YORK,NY


In [20]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3://information-arch-vendor-bucket/' #specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_all_pages' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://information-arch-vendor-bucket/charities_bureau_scrape_all_pages20240413040229.csv


## References
* https://www.programiz.com/python-programming/working-csv-files
* https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
* https://realpython.com/python-boto3-aws-s3/
* https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 